<a href="https://colab.research.google.com/github/CristhianSeverino/Architecture_A/blob/main/ERP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Install Libraries**📚

*In this Section*, We intall and import the libraries





In [ ]:
!pip install faker
print("="*70)
print(" "*15 + "Lybraries installed 😉")
print("="*70)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 11.7 MB/s eta 0:00:00
               Lybraries installed 😉


In [ ]:
import pandas as pd
import numpy as np
from faker import Faker
import random
print("="*70)
print(" "*15 + " Lybraries imported 😉")
print("="*70)

                Lybraries imported 😉


# **Syntetic Data**
In this section. **We create the funtion sintetyc data**. the objetive is simulated a ERP sistem (Accounting system).

We based in Star schema. We think in End- to- end data solution.

In [12]:
def generate_star_schema(num_fact_rows=50000):
    fake = Faker('es_CO')

    # 1. Dimension: Customer (Optimización de IDs únicos)
    num_customers = 10000
    # Generamos IDs únicos de una sola vez para evitar el loop 'while' y colisiones
    customer_ids = np.random.choice(np.arange(1000000000, 1000100000), num_customers, replace=False)

    dim_customer = pd.DataFrame({
        'customer_key': np.arange(num_customers), # Surrogate Key
        'customer_id': customer_ids,
        'name': [fake.name() for _ in range(num_customers)],
        'category': np.random.choice(['BtoB', 'BtoC'], num_customers),
        'email': [fake.email() for _ in range(num_customers)],
        'birth_date': [fake.date_of_birth(minimum_age=18, maximum_age=45) for _ in range(num_customers)],
        'phone': [fake.phone_number() for _ in range(num_customers)],
    })

    # 2. Dimension: Product
    num_products = 350
    dim_product = pd.DataFrame({
        'product_id': np.arange(num_products),
        'product_name': [fake.catch_phrase() for _ in range(num_products)],
        'category': np.random.choice(['Electronics', 'Apparel', 'Toys'], num_products),
        'unit_price': np.round(np.random.uniform(5.0, 500.0, num_products), 2),
        'stock_code': [f"PROD{i+1:04d}" for i in range(num_products)]
    })

    # 3. Dimension: Time (Vectorizado con Pandas)
    dates = pd.date_range('2024-01-01', '2024-12-31', freq='D')
    dim_time = pd.DataFrame({
        'date_id': np.arange(len(dates)),
        'date': dates,
        'year': dates.year,
        'month': dates.month,
        'day': dates.day,
        'day_of_week': dates.day_name(),
    })

    # 4. Dimension: Location
    cities = ['Manizales', 'Bogota', 'Pereira', 'Medellin', 'Cartagena', 'Barranquilla', 'Santa Marta', 'Neiva', 'Mitu', 'Valledupar', 'Mocoa']
    dim_location = pd.DataFrame({
        'location_id': np.arange(len(cities)),
        'city_name': cities,
        'address': [fake.address() for _ in range(len(cities))],
    })

    # 5. Fact Table: Sales (Generación Vectorizada Masiva)
    # Usamos random.choices o np.random para velocidad
    fact_sales = pd.DataFrame({
        'sale_id': np.arange(num_fact_rows),
        'customer_key': np.random.choice(dim_customer['customer_key'], num_fact_rows),
        'product_id': np.random.randint(0, num_products, num_fact_rows),
        'date_id': np.random.randint(0, len(dates), num_fact_rows),
        'location_id': np.random.randint(0, len(cities), num_fact_rows),
        'quantity_sold': np.random.randint(1, 11, num_fact_rows),
    })

    # Join eficiente para calcular ingresos
    fact_sales = fact_sales.merge(dim_product[['product_id', 'unit_price']], on='product_id', how='left')
    fact_sales['total_revenue'] = (fact_sales['unit_price'] * fact_sales['quantity_sold']).round(2)

    return {
        'fact_sales': fact_sales.drop(columns=['unit_price']),
        'dim_customer': dim_customer,
        'dim_product': dim_product,
        'dim_time': dim_time,
        'dim_location': dim_location
    }

# **Data Generation**

> Creation syntetic data.



In [ ]:
#========================================== Function Execution =======================================

# 1. Generate the DataFrames with the `generate_star_schema` function
print("Starting the generation of business data... 📈")

# You can adjust the number of rows in the fact table here
tables_business = generate_star_schema(num_fact_rows=100000)

print("Data generated successfully. 🎉")


print("Full charge. ✅")

Starting the generation of business data... 📈
Data generated successfully. 🎉
Full charge. ✅


In [ ]:
tables_business['fact_sales'].head()

,sale_id,customer_key,product_id,date_id,location_id,quantity_sold,total_revenue
0,0,9199,307,257,6,8,1331.84
1,1,7522,211,233,7,5,325.60
2,2,9753,40,97,10,9,1262.88
3,3,5991,36,194,9,4,1316.28
4,4,4516,120,95,7,3,118.68


In [ ]:
tables_business['dim_customer'].head()

,customer_key,customer_id,name,category,email,birth_date,phone
0,0,1000010443,Óscar Páez Márquez,BtoB,ordonezcristina@example.org,1982-05-04,2850101
1,1,1000022541,Aida Pérez,BtoC,wlopez@example.org,1992-07-22,+57 311 726 65 99
2,2,1000032786,Nelson Arroyo Cantillo,BtoC,wtorres@example.com,1990-06-23,57 323 672 10 75
3,3,1000074485,Deyanira Saavedra,BtoC,luis21@example.org,1988-02-10,3112686511
4,4,1000077029,Rosa García,BtoB,amosquera@example.net,1999-10-11,+57 607 376 24 34


In [ ]:
tables_business['dim_customer'].head()


,customer_key,customer_id,name,category,email,birth_date,phone
0,0,1000010443,Óscar Páez Márquez,BtoB,ordonezcristina@example.org,1982-05-04,2850101
1,1,1000022541,Aida Pérez,BtoC,wlopez@example.org,1992-07-22,+57 311 726 65 99
2,2,1000032786,Nelson Arroyo Cantillo,BtoC,wtorres@example.com,1990-06-23,57 323 672 10 75
3,3,1000074485,Deyanira Saavedra,BtoC,luis21@example.org,1988-02-10,3112686511
4,4,1000077029,Rosa García,BtoB,amosquera@example.net,1999-10-11,+57 607 376 24 34


In [ ]:
print("---"*30)
tables_business['dim_product'].head()


------------------------------------------------------------------------------------------


,product_id,product_name,category,unit_price,stock_code
0,0,Visionary maximized implementation,Toys,284.67,PROD0001
1,1,Balanced interactive capacity,Electronics,452.62,PROD0002
2,2,Monitored bottom-line leverage,Electronics,45.81,PROD0003
3,3,Team-oriented transitional data-warehouse,Toys,220.00,PROD0004
4,4,Decentralized bifurcated structure,Toys,75.58,PROD0005


In [ ]:
print("---"*30)
tables_business['dim_time'].head()


------------------------------------------------------------------------------------------


,date_id,date,year,month,day,day_of_week
0,0,2024-01-01,2024,1,1,Monday
1,1,2024-01-02,2024,1,2,Tuesday
2,2,2024-01-03,2024,1,3,Wednesday
3,3,2024-01-04,2024,1,4,Thursday
4,4,2024-01-05,2024,1,5,Friday


In [ ]:
print("---"*30)
tables_business['dim_location'].head()

------------------------------------------------------------------------------------------


,location_id,city_name,address
0,0,Manizales,"Dg. 38F Este # 2-74 Este\n179998\nPalestina, C..."
1,1,Bogota,"Av. Herrán # 77-9 Este\n152034\nSamacá, Boyacá"
2,2,Pereira,Diagonal 92 # 60-56\nLocal 73\n853386\nChámeza...
3,3,Medellin,Avenida García # 97-83 Sur\nBodega 5\n232925\n...
4,4,Cartagena,Avenida calle 155 # 8-1\nBodega 35\n663500\nDo...


In [ ]:
tables_business['fact_sales'].to_csv('fact_sales.csv', index=False)
tables_business['dim_customer'].to_csv('dim_customer.csv', index=False)
tables_business['dim_product'].to_csv('dim_product.csv', index=False)
tables_business['dim_time'].to_csv('dim_time.csv', index=False)
tables_business['dim_location'].to_csv('dim_location.csv', index=False)


In [13]:
from google.colab import files
files.download('fact_sales.csv')
files.download('dim_customer.csv')
files.download('dim_product.csv')
files.download('dim_time.csv')
files.download('dim_location.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>